In [1]:
'''
PYTORCH-101 
CIFAR10 with VGG16 model. Written by Gicheon Kang(@gicheonkang)
MIT LICENSE
'''

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import collections
import math


#hyper parameters
batch_size = 100
learning_rate = 0.001
num_class = 10
num_epoch = 10

# cifar10 dataset
train_dataset = dsets.CIFAR10(root='./data', 
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = dsets.CIFAR10(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        # network architecture
        self.network = nn.Sequential(
                            nn.Conv2d(3, 64, kernel_size=3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.BatchNorm2d(64),
                            nn.Conv2d(64, 64, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(64),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            
                            nn.Conv2d(64, 128, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(128),
                            nn.Conv2d(128, 128, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(128),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            
                            nn.Conv2d(128, 256, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(256),
                            nn.Conv2d(256, 256, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(256),
                            nn.Conv2d(256, 256, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(256),
                            nn.MaxPool2d(kernel_size=2, stride=2),
            
                            nn.Conv2d(256, 512, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(512),
                            nn.Conv2d(512, 512, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(512),
                            nn.Conv2d(512, 512, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(512),
                            nn.MaxPool2d(kernel_size=2, stride=2),
            
                            nn.Conv2d(512, 512, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(512),
                            nn.Conv2d(512, 512, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(512),
                            nn.Conv2d(512, 512, kernel_size=3, padding=1),
                            nn.ReLU(True),
                            nn.BatchNorm2d(512),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                        )
        
        # fully-connected layer
        self.classifier = nn.Sequential(
                            nn.Linear(512, 512),
                            nn.ReLU(True),
                            nn.Linear(512, 512),
                            nn.ReLU(True),
                            nn.Linear(512, 10),
                        )    
    
    def forward(self, image):
        x = self.network(image)
        x = x.view(-1, 512 * 1 * 1)
        out = self.classifier(x)
        return out

In [12]:
MODEL = VGG16()
print (MODEL)
MODEL.cuda(0)
criterion = nn.CrossEntropyLoss()
optimization = optim.Adam(MODEL.parameters(), lr=learning_rate)

VGG16(
  (network): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): BatchNorm2d(256, eps=1e-05, 

In [9]:
# training step
for epoch in range(num_epoch):
    for i, (image, label) in enumerate(train_loader):
        images = Variable(image.cuda())
        labels = Variable(label.cuda())
        
        optimization.zero_grad()
        output = MODEL(images)
        loss = criterion(output, labels)
        loss.backward()
        optimization.step()
        
        if(i+1) % 100 == 0:
            print('epoch: [%d/%d], step: [%d/%d], Loss: %.4f' % (epoch+1, num_epoch, i+1, len(train_dataset)//batch_size, loss.data[0]))

epoch: [1/10], step: [100/500], Loss: 1.6252
epoch: [1/10], step: [200/500], Loss: 1.5621
epoch: [1/10], step: [300/500], Loss: 1.3278
epoch: [1/10], step: [400/500], Loss: 1.2692
epoch: [1/10], step: [500/500], Loss: 1.3732
epoch: [2/10], step: [100/500], Loss: 0.9383
epoch: [2/10], step: [200/500], Loss: 0.9118
epoch: [2/10], step: [300/500], Loss: 1.0744
epoch: [2/10], step: [400/500], Loss: 0.8170
epoch: [2/10], step: [500/500], Loss: 0.6352
epoch: [3/10], step: [100/500], Loss: 0.7141
epoch: [3/10], step: [200/500], Loss: 0.7108
epoch: [3/10], step: [300/500], Loss: 0.9597
epoch: [3/10], step: [400/500], Loss: 0.8471
epoch: [3/10], step: [500/500], Loss: 0.6029
epoch: [4/10], step: [100/500], Loss: 0.8042
epoch: [4/10], step: [200/500], Loss: 0.7466
epoch: [4/10], step: [300/500], Loss: 0.5192
epoch: [4/10], step: [400/500], Loss: 0.5352
epoch: [4/10], step: [500/500], Loss: 0.5464
epoch: [5/10], step: [100/500], Loss: 0.5411
epoch: [5/10], step: [200/500], Loss: 0.4508
epoch: [5/

In [10]:
total = 0
correct = 0

for (image, label) in test_loader:
    images = Variable(image.cuda())
    
    output = MODEL(images)
    _, predicted = torch.max(output.data, 1)
    correct += (predicted.cpu()==label).sum()
    total += labels.size(0)

print('Accuracy : %d%%' %((100*correct)/total))

Accuracy : 96%
